## 데이터 전처리
### 기본 설정

In [1]:
# 패키지 로드
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from collections import defaultdict
import os, random

from scipy import sparse
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
from torch.nn.init import normal_
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

import plotnine
from plotnine import *

import pickle

In [2]:
# 경로 설정
data_path = './data'
saved_path = './code/saved'
output_path = './code/submission'

### 데이터 불러오기
- history_data : 시청 시작 데이터
- profile_data : 프로필 정보 
- meta_data : 콘텐츠 일반 메타 정보

In [3]:
# 데이터 불러오기 
history_df = pd.read_csv(os.path.join(data_path, 'history_data.csv'), encoding='utf-8')
search_df = pd.read_csv(os.path.join(data_path, 'search_data.csv'), encoding='utf-8')
profile_df = pd.read_csv(os.path.join(data_path, 'profile_data.csv'), encoding='utf-8')
meta_df = pd.read_csv(os.path.join(data_path, 'meta_data.csv'), encoding='utf-8')
we_df = pd.read_csv(os.path.join(data_path, 'watch_e_data.csv'), encoding='utf-8')
buy_df = pd.read_csv(os.path.join(data_path, 'buy_data.csv'), encoding='utf-8')

## 앨범 관심 시청별 최다 키워드

In [4]:
### 키워드 별 인원을 나타내기위해 하나로 모으기
keyword1 = profile_df[['profile_id', 'pr_interest_keyword_cd_1', 'ch_interest_keyword_cd_1']]
keyword2 = profile_df[['profile_id', 'pr_interest_keyword_cd_2', 'ch_interest_keyword_cd_2']]
keyword3 = profile_df[['profile_id', 'pr_interest_keyword_cd_3', 'ch_interest_keyword_cd_3']]

In [5]:
### cbind하기 전 이름 통일
keyword1.columns = ["profile_id","pr_int","ch_int"]
keyword2.columns = ["profile_id","pr_int","ch_int"]
keyword3.columns = ["profile_id","pr_int","ch_int"]

In [6]:
### 위/아래로 합치기 - 행 기준
keyword = pd.concat([keyword1, keyword2, keyword3], axis = 0).dropna(axis=0)

In [7]:
keyword

,profile_id,pr_int,ch_int
0,3,P02,K01
1,5,P07,K05
2,7,P05,K06
3,12,P03,K09
4,16,P03,K01
...,...,...,...
8305,33019,P01,K09
8307,33023,P07,K05
8308,33026,P08,K06
8309,33027,P06,K05


In [9]:
### 행동 패턴 데이터
data = pd.read_csv("behavior.csv")
### 모든 데이터의 시작은 1로
data['start'] = 1
### 종료 1인 경우 양수 0인 경우 음수
data['sign'] = 1
data['sign'][data['yes_watch'] == 0] = -1
data['yes_watch'] = data['yes_watch'] - 0.5
data['score'] = (2.5 + data['yes_watch'] + (data['buy'] + data['search'])*data['sign'])/15

In [14]:
data

,profile_id,album_id,yes_watch,payment,buy,search,start,sign,score
0,3,15,0.5,0.0,0.0,0.0,1,1,0.2
1,3,16,0.5,0.0,0.0,0.0,1,1,0.2
2,3,16,0.5,0.0,0.0,0.0,1,1,0.2
3,3,17,0.5,0.0,0.0,0.0,1,1,0.2
4,3,18,0.5,0.0,0.0,0.0,1,1,0.2
...,...,...,...,...,...,...,...,...,...
752742,33032,381,0.5,0.0,0.0,0.0,1,1,0.2
752743,33032,375,0.5,0.0,0.0,0.0,1,1,0.2
752744,33032,375,0.5,0.0,0.0,0.0,1,1,0.2
752745,33032,125,0.5,0.0,0.0,0.0,1,1,0.2


In [13]:
data.drop_duplicates(['profile_id', 'album_id'])

,profile_id,album_id,yes_watch,payment,buy,search,start,sign,score
0,3,15,0.5,0.0,0.0,0.0,1,1,0.200000
1,3,16,0.5,0.0,0.0,0.0,1,1,0.200000
3,3,17,0.5,0.0,0.0,0.0,1,1,0.200000
4,3,18,0.5,0.0,0.0,0.0,1,1,0.200000
6,3,19,0.5,0.0,0.0,0.0,1,1,0.200000
...,...,...,...,...,...,...,...,...,...
752732,33032,190,-0.5,0.0,0.0,0.0,1,-1,0.133333
752733,33032,381,0.5,0.0,0.0,0.0,1,1,0.200000
752739,33032,327,-0.5,0.0,0.0,0.0,1,-1,0.133333
752740,33032,1725,-0.5,0.0,0.0,0.0,1,-1,0.133333


In [7]:
new = we_df[['profile_id', 'album_id']].drop_duplicates(['profile_id', 'album_id'])

In [21]:
new

,profile_id,album_id
0,3,15
1,3,16
2,3,17
3,3,18
4,3,19
...,...,...
892777,33032,0
892778,33032,1725
892780,33032,3128
892782,33032,3625


In [9]:
new_meta = meta_df.drop_duplicates('album_id')[['album_id', 'genre_mid']]

In [10]:
new_k = pd.merge(new, new_meta, on = 'album_id', how = 'left')

In [256]:
new_k

,profile_id,album_id,genre_mid
0,3,15,노래율동
1,3,16,노래율동
2,3,17,노래율동
3,3,18,노래율동
4,3,19,노래율동
...,...,...,...
347977,33032,0,외국어
347978,33032,1725,노래율동
347979,33032,3128,외국어
347980,33032,3625,노래율동


In [258]:
kk = pd.merge(new_k, keyword, on = 'profile_id', how = 'inner')

In [260]:
kk

,profile_id,album_id,genre_mid,pr_int,ch_int
0,3,15,노래율동,P02,K01
1,3,15,노래율동,P04,K03
2,3,15,노래율동,P07,K04
3,3,16,노래율동,P02,K01
4,3,16,노래율동,P04,K03
...,...,...,...,...,...
845738,33032,3625,노래율동,P05,K05
845739,33032,3625,노래율동,P07,K09
845740,33032,375,놀이교실,P03,K01
845741,33032,375,놀이교실,P05,K05


In [11]:
k = pd.merge(new, keyword, on = 'profile_id', how = 'inner')

In [24]:
k

,profile_id,album_id,pr_int,ch_int
0,3,15,P02,K01
1,3,15,P04,K03
2,3,15,P07,K04
3,3,16,P02,K01
4,3,16,P04,K03
...,...,...,...,...
845738,33032,3625,P05,K05
845739,33032,3625,P07,K09
845740,33032,375,P03,K01
845741,33032,375,P05,K05


In [12]:
### 키워드 별 인원수 세기
kpcount = keyword.groupby('pr_int')['profile_id'].count()
kccount = keyword.groupby('ch_int')['profile_id'].count()

In [26]:
kpcount

pr_int
P01    2102
P02    3112
P03    2881
P04    1773
P05    1837
P06    3780
P07    3238
P08    1560
Name: profile_id, dtype: int64

In [273]:
### 부모, 자녀 키워드 앨범, 키워드별 각각 인원수 세기
kk2 = pd.DataFrame(kk.groupby(['genre_mid','pr_int']).count()).reset_index()
kk3 = pd.DataFrame(kk.groupby(['genre_mid','ch_int']).count()).reset_index()

In [279]:
kk2

,genre_mid,pr_int,profile_id,album_id,ch_int
0,MCN,P01,160,160,160
1,MCN,P02,84,84,84
2,MCN,P03,115,115,115
3,MCN,P04,50,50,50
4,MCN,P05,57,57,57
...,...,...,...,...,...
205,한자,P02,2,2,2
206,한자,P03,3,3,3
207,한자,P06,1,1,1
208,한자,P07,2,2,2


In [288]:
### 키워드를 열로 옮기기
par_with_genre_cnt = kk2.pivot(
    index = 'genre_mid',
    columns = 'pr_int',
    values = 'profile_id'
)

chi_with_genre_cnt = kk3.pivot(
    index = 'genre_mid',
    columns = 'ch_int',
    values = 'profile_id'
)

### 없는값은 0으로 채우기
### 앨범 별 키워드 인원 
par_with_genre_cnt = par_with_genre_cnt.reset_index().fillna(0)
chi_with_genre_cnt = chi_with_genre_cnt.reset_index().fillna(0)

In [292]:
kpcount

pr_int
P01    2102
P02    3112
P03    2881
P04    1773
P05    1837
P06    3780
P07    3238
P08    1560
Name: profile_id, dtype: int64

In [294]:
par_with_genre_cnt

pr_int,genre_mid,P01,P02,P03,P04,P05,P06,P07,P08
0,MCN,0.076118,0.026992,0.039917,0.028201,0.031029,0.033862,0.028104,0.038462
1,TV만화,8.976689,11.637853,10.658799,13.338973,13.187262,10.644444,11.032119,8.969872
2,극장판 애니,0.016651,0.024100,0.022909,0.025945,0.023408,0.018519,0.019148,0.012821
3,노래 율동,0.111323,0.197301,0.197501,0.213762,0.255308,0.202381,0.194873,0.162179
4,노래율동,6.609420,15.146208,13.665741,17.170333,17.504627,15.066931,14.109944,7.364103
5,놀이교실,3.694101,3.394602,3.992711,4.219402,3.862275,3.672751,3.901174,3.941026
6,다큐멘터리,0.028069,0.025064,0.033322,0.016356,0.015787,0.023016,0.026868,0.028205
7,독서동화,0.103711,0.138817,0.124262,0.142696,0.091998,0.129365,0.160593,0.125641
8,드라마,0.050904,0.025064,0.032975,0.033841,0.036473,0.043122,0.037678,0.050000
9,수학과학,0.039010,0.032134,0.035751,0.021433,0.027763,0.024603,0.048796,0.076282


In [293]:
### 선택 인원수 별로 나누어줌
for i in range(8):
    par_with_genre_cnt.iloc[:,i+1] = par_with_genre_cnt.iloc[:,i+1]/kpcount[i]
    
for j in range(9):
    chi_with_genre_cnt.iloc[:,j+1] = chi_with_genre_cnt.iloc[:,j+1]/kccount[j]

In [295]:
### 다시 melt
par_with_genre_melt = par_with_genre_cnt.melt(
    id_vars = "genre_mid",
    value_vars = ['P01','P02','P03','P04','P05','P06','P07','P08'],
    value_name = 'viewpersel',
    var_name = "pr_int"
)

chi_with_genre_melt = chi_with_genre_cnt.melt(
    id_vars = "genre_mid",
    value_vars = ['K01','K02','K03','K04','K05','K06','K07','K08','K09'],
    value_name = 'viewpersel',
    var_name = "ch_int"
)

In [296]:
par_with_genre_melt

,genre_mid,pr_int,viewpersel
0,MCN,P01,0.076118
1,TV만화,P01,8.976689
2,극장판 애니,P01,0.016651
3,노래 율동,P01,0.111323
4,노래율동,P01,6.609420
...,...,...,...
219,책,P08,10.410897
220,체육안전,P08,0.005128
221,코미디,P08,0.070513
222,한글,P08,0.072436


In [297]:
### 각 앨범별 최대값과 그에 해당하는 선택 키워드
par_with_genre_max = par_with_genre_melt.loc[par_with_genre_melt.groupby(['genre_mid'])['viewpersel'].idxmax()]
chi_with_genre_max = chi_with_genre_melt.loc[chi_with_genre_melt.groupby(['genre_mid'])['viewpersel'].idxmax()]

In [312]:
pr_genre_dict = par_with_genre_max[['genre_mid', 'pr_int']].set_index('genre_mid').to_dict()
ch_genre_dict = chi_with_genre_max[['genre_mid', 'ch_int']].set_index('genre_mid').to_dict()

In [313]:
pr_genre_dict

{'pr_int': {'MCN': 'P01',
  'TV만화': 'P04',
  '극장판 애니': 'P04',
  '노래 율동': 'P05',
  '노래율동': 'P05',
  '놀이교실': 'P04',
  '다큐멘터리': 'P03',
  '독서동화': 'P07',
  '드라마': 'P01',
  '수학과학': 'P08',
  '시리즈': 'P04',
  '시사교양': 'P01',
  '애니': 'P05',
  '액션/모험': 'P04',
  '어린이방송': 'P01',
  '역사/문화': 'P03',
  '연예오락': 'P05',
  '예능': 'P08',
  '예술': 'P03',
  '외국어': 'P08',
  '유아애니': 'P04',
  '육아정보': 'P05',
  '창의학습': 'P08',
  '책': 'P08',
  '체육안전': 'P01',
  '코미디': 'P04',
  '한글': 'P08',
  '한자': 'P03'}}

In [316]:
ch_genre_dict['ch_int']['TV만화']

'K08'

### 장르 결합

In [13]:
### 부모, 자녀 키워드 앨범, 키워드별 각각 인원수 세기
k2 = pd.DataFrame(k.groupby(['album_id','pr_int']).count()).reset_index()
k3 = pd.DataFrame(k.groupby(['album_id','ch_int']).count()).reset_index()

In [14]:
### 키워드를 열로 옮기기
par_cnt = k2.pivot(
    index = 'album_id',
    columns = 'pr_int',
    values = 'profile_id'
)

chi_cnt = k3.pivot(
    index = 'album_id',
    columns = 'ch_int',
    values = 'profile_id'
)

In [15]:
### 없는값은 0으로 채우기
### 앨범 별 키워드 인원 
par_cnt = par_cnt.reset_index().fillna(0)
chi_cnt = chi_cnt.reset_index().fillna(0)

In [16]:
### 선택 인원수 별로 나누어줌
for i in range(8):
    par_cnt.iloc[:,i+1] = par_cnt.iloc[:,i+1]/kpcount[i]
    
for j in range(9):
    chi_cnt.iloc[:,j+1] = chi_cnt.iloc[:,j+1]/kccount[j]

In [17]:
par_cnt

pr_int,album_id,P01,P02,P03,P04,P05,P06,P07,P08
0,0,0.077545,0.090296,0.093023,0.089114,0.087099,0.097354,0.106547,0.094231
1,1,0.011893,0.018316,0.017355,0.018613,0.016875,0.018519,0.017603,0.013462
2,2,0.011893,0.014781,0.017008,0.012972,0.013609,0.016931,0.016986,0.016026
3,3,0.014272,0.012532,0.014578,0.015792,0.018508,0.010847,0.016677,0.010897
4,4,0.008563,0.017031,0.017008,0.024253,0.018508,0.014286,0.016059,0.014744
...,...,...,...,...,...,...,...,...,...
20356,25912,0.000476,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
20357,25913,0.000476,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
20358,25914,0.000476,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
20359,25915,0.000476,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [18]:
### 다시 melt
par_melt = par_cnt.melt(
    id_vars = "album_id",
    value_vars = ['P01','P02','P03','P04','P05','P06','P07','P08'],
    value_name = 'viewpersel',
    var_name = "pr_int"
)

chi_melt = chi_cnt.melt(
    id_vars = "album_id",
    value_vars = ['K01','K02','K03','K04','K05','K06','K07','K08','K09'],
    value_name = 'viewpersel',
    var_name = "ch_int"
)

In [19]:
par_melt

,album_id,pr_int,viewpersel
0,0,P01,0.077545
1,1,P01,0.011893
2,2,P01,0.011893
3,3,P01,0.014272
4,4,P01,0.008563
...,...,...,...
162883,25912,P08,0.000000
162884,25913,P08,0.000000
162885,25914,P08,0.000000
162886,25915,P08,0.000000


In [32]:
### 각 앨범별 최대값과 그에 해당하는 선택 키워드
par_max = par_melt.loc[par_melt.groupby(['album_id'])['viewpersel'].idxmax()]
chi_max = chi_melt.loc[chi_melt.groupby(['album_id'])['viewpersel'].idxmax()]

## 장르와 융합

In [67]:
### 중장르 뽑기
genre = meta_df[['album_id', 'genre_mid', 'sub_title']]

In [68]:
genre = genre.drop_duplicates('album_id')

In [69]:
genre

,album_id,genre_mid,sub_title
0,749,TV만화,꼬마버스 타요1
1,750,TV만화,꼬마버스 타요1
2,2131,TV만화,꼬마버스 타요1
3,2625,TV만화,꼬마버스 타요1
4,2594,TV만화,꼬마버스 타요1
...,...,...,...
42596,39872,놀이교실,로티프렌즈 미술놀이
42597,39873,놀이교실,로티프렌즈 미술놀이
42598,39874,놀이교실,로티프렌즈 미술놀이
42599,4779,책,4-5세


In [51]:
meta_df.album_id.nunique()

39875

In [52]:
par_fu.album_id.nunique()

20361

In [ ]:
me

In [55]:
chi_fu.album_id.nunique()

20361

In [299]:
genre

,album_id,genre_mid,sub_title
0,749,TV만화,꼬마버스 타요1
1,750,TV만화,꼬마버스 타요1
2,2131,TV만화,꼬마버스 타요1
3,2625,TV만화,꼬마버스 타요1
4,2594,TV만화,꼬마버스 타요1
...,...,...,...
42596,39872,놀이교실,로티프렌즈 미술놀이
42597,39873,놀이교실,로티프렌즈 미술놀이
42598,39874,놀이교실,로티프렌즈 미술놀이
42599,4779,책,4-5세


In [301]:
### 부모, 자녀 각 키워드랑 융합
par_with_genre_fu = pd.merge(genre, par_with_genre_max[['genre_mid','pr_int']], on = 'genre_mid')
chi_with_genre_fu = pd.merge(genre, chi_with_genre_max[['genre_mid','ch_int']], on = 'genre_mid')

In [303]:
par_with_genre_fu

,album_id,genre_mid,sub_title,pr_int
0,749,TV만화,꼬마버스 타요1,P04
1,750,TV만화,꼬마버스 타요1,P04
2,2131,TV만화,꼬마버스 타요1,P04
3,2625,TV만화,꼬마버스 타요1,P04
4,2594,TV만화,꼬마버스 타요1,P04
...,...,...,...,...
39850,6302,한글,EBS 초등 만점왕 - 국어 2-1,P08
39851,6303,한글,EBS 초등 만점왕 - 국어 2-1,P08
39852,6309,한글,EBS 초등 만점왕 - 국어 2-1,P08
39853,6310,한글,EBS 초등 만점왕 - 국어 2-1,P08


In [304]:
chi_with_genre_fu

,album_id,genre_mid,sub_title,ch_int
0,749,TV만화,꼬마버스 타요1,K08
1,750,TV만화,꼬마버스 타요1,K08
2,2131,TV만화,꼬마버스 타요1,K08
3,2625,TV만화,꼬마버스 타요1,K08
4,2594,TV만화,꼬마버스 타요1,K08
...,...,...,...,...
39850,6302,한글,EBS 초등 만점왕 - 국어 2-1,K05
39851,6303,한글,EBS 초등 만점왕 - 국어 2-1,K05
39852,6309,한글,EBS 초등 만점왕 - 국어 2-1,K05
39853,6310,한글,EBS 초등 만점왕 - 국어 2-1,K05


In [70]:
### 부모, 자녀 각 키워드랑 융합
par_fu = pd.merge(genre, par_max[['album_id','pr_int']],left_on='album_id', right_on='album_id', how='right')
chi_fu = pd.merge(genre, chi_max[['album_id','ch_int']],left_on='album_id', right_on='album_id', how='right')

In [71]:
par_fu

,album_id,genre_mid,sub_title,pr_int
0,0,외국어,디즈니 프린세스,P07
1,1,외국어,베베 뮤직스토리,P04
2,2,외국어,베베 뮤직스토리,P03
3,3,외국어,픽사,P05
4,4,TV만화,출동! 슈퍼윙스1,P04
...,...,...,...,...
20356,25912,독서동화,주니토니 NEW 동화뮤지컬,P01
20357,25913,노래율동,퓨처북 공룡 동요2,P01
20358,25914,노래율동,퓨처북 공룡 동요2,P01
20359,25915,노래율동,퓨처북 공룡 동요2,P01


In [317]:
par_fu.groupby(['sub_title']).value_counts().reset_index()

,sub_title,album_id,genre_mid,pr_int,0
0,100분! 뽀요 인기 메들리,23104,노래율동,P03,1
1,10월 세계 여러나라,6596,놀이교실,P04,1
2,11월 지구와 우주,7071,놀이교실,P08,1
3,12월 겨울,2633,놀이교실,P08,1
4,1월 생활도구,3806,놀이교실,P07,1
...,...,...,...,...,...
20356,히어로 써클,22399,TV만화,P04,1
20357,히어로 써클,22397,TV만화,P04,1
20358,히어로 써클,21564,TV만화,P04,1
20359,히어로 써클,16177,TV만화,P04,1


In [63]:
meta_sub_title = meta_df.sub_title.value_counts().reset_index()
meta_sub_title.columns = ['sub_title', 'count']

In [86]:
# sub_title 하나인 것 -> 164개
meta_sub_title.loc[meta_sub_title['count'] == 1]

#sub_title 두개인 것 -> 99개
meta_sub_title.loc[meta_sub_title['count'] == 2]

#sub_title 3개인 것 -> 91개
meta_sub_title.loc[meta_sub_title['count'] == 1]

,sub_title,count
2209,"오늘의 영어 3주차 알파벳 ""wh, th, ck"" / Community",1
2210,Jesse의 플레이 키친(하),1
2211,"오늘의 영어 13주차 파닉스 ""ea (I, e)"" / Education",1
2212,도전 에코 키즈,1
2213,"오늘의 영어 9주차 알파벳 ""ar, er"" / Special Days",1
...,...,...
2368,12월 겨울,1
2369,1월 생활도구,1
2370,2월 자연과 동물,1
2371,3월 새로움이 많아요,1


In [321]:
meta_genre = meta_df.drop_duplicates('album_id')[['album_id', 'genre_mid']]

In [322]:
meta_genre

,album_id,genre_mid
0,749,TV만화
1,750,TV만화
2,2131,TV만화
3,2625,TV만화
4,2594,TV만화
...,...,...
42596,39872,놀이교실
42597,39873,놀이교실
42598,39874,놀이교실
42599,4779,책


In [356]:
pr_genre_dict['pr_int']

{'MCN': 'P01',
 'TV만화': 'P04',
 '극장판 애니': 'P04',
 '노래 율동': 'P05',
 '노래율동': 'P05',
 '놀이교실': 'P04',
 '다큐멘터리': 'P03',
 '독서동화': 'P07',
 '드라마': 'P01',
 '수학과학': 'P08',
 '시리즈': 'P04',
 '시사교양': 'P01',
 '애니': 'P05',
 '액션/모험': 'P04',
 '어린이방송': 'P01',
 '역사/문화': 'P03',
 '연예오락': 'P05',
 '예능': 'P08',
 '예술': 'P03',
 '외국어': 'P08',
 '유아애니': 'P04',
 '육아정보': 'P05',
 '창의학습': 'P08',
 '책': 'P08',
 '체육안전': 'P01',
 '코미디': 'P04',
 '한글': 'P08',
 '한자': 'P03'}

In [379]:
meta_genre_with_keyowrd = pd.merge(meta_genre, par_with_genre_max[['genre_mid','pr_int']], on = 'genre_mid', how = 'left')

In [380]:
meta_genre_with_keyowrd

,album_id,genre_mid,pr_int
0,749,TV만화,P04
1,750,TV만화,P04
2,2131,TV만화,P04
3,2625,TV만화,P04
4,2594,TV만화,P04
...,...,...,...
39870,39872,놀이교실,P04
39871,39873,놀이교실,P04
39872,39874,놀이교실,P04
39873,4779,책,P08


In [92]:
h = meta_df.drop_duplicates('album_id')[['album_id', 'sub_title']]

In [100]:
h

,album_id,sub_title
0,749,꼬마버스 타요1
1,750,꼬마버스 타요1
2,2131,꼬마버스 타요1
3,2625,꼬마버스 타요1
4,2594,꼬마버스 타요1
...,...,...
42596,39872,로티프렌즈 미술놀이
42597,39873,로티프렌즈 미술놀이
42598,39874,로티프렌즈 미술놀이
42599,4779,4-5세


In [146]:
new_par_fu = pd.merge(h, par_max[['album_id','pr_int']],left_on='album_id', right_on='album_id', how='left')

In [114]:
new_par_fu

,album_id,sub_title,pr_int
0,749,꼬마버스 타요1,P05
1,750,꼬마버스 타요1,P05
2,2131,꼬마버스 타요1,P05
3,2625,꼬마버스 타요1,P05
4,2594,꼬마버스 타요1,P05
...,...,...,...
39870,39872,로티프렌즈 미술놀이,NaN
39871,39873,로티프렌즈 미술놀이,NaN
39872,39874,로티프렌즈 미술놀이,NaN
39873,4779,4-5세,NaN


In [242]:
ho = new_par_fu[['sub_title', 'pr_int']].value_counts().reset_index().sort_values('sub_title')

In [135]:
meta_subtitle_key_dict = dict()

temp = 'a'
MAX = 0
for i in ho.itertuples():
    if temp == i[1]:
        MAX = max(MAX, i[3])
        if MAX == i[3]:
            meta_subtitle_key_dict[i[1]] = i[2]
    else:
        temp = i[1]
        meta_subtitle_key_dict[i[1]] = i[2]
        

In [136]:
meta_subtitle_key_dict

{'100분! 뽀요 인기 메들리': 'P03',
 '10월 세계 여러나라': 'P04',
 '11월 지구와 우주': 'P08',
 '12월 겨울': 'P08',
 '1월 생활도구': 'P07',
 '2월 자연과 동물': 'P04',
 '3세이하': 'P02',
 '3월 새로움이 많아요': 'P04',
 '4-5세': 'P01',
 '4남매쇼': 'P06',
 '4월 봄': 'P04',
 '5가지 뽀롱뽀롱 꿈 상담소': 'P02',
 '5분 종이접기 한반도의 공룡편': 'P04',
 '5월 몸짱 마음짱': 'P08',
 '6-7세': 'P01',
 '60분 뽀요 율동 메들리': 'P02',
 '60초툰': 'P04',
 '6월 우리동네': 'P04',
 '7월 여름과 건강': 'P04',
 '8-9세': 'P01',
 '8월 교통기관': 'P04',
 '9월 우리나라': 'P04',
 'A Day with Little Monkey': 'P06',
 'A New Baby Is Coming': 'P08',
 'A Picnic Day': 'P08',
 'A Super Halloween': 'P04',
 'A Wish For Whales': 'P03',
 'ABC Song': 'P08',
 'Alligator Eats Candies': 'P08',
 'Animals at the Zoo': 'P03',
 'A~Z까지 알파벳 배우기': 'P08',
 'Baby Bear`s New Friend': 'P07',
 'Bath Time Fun': 'P08',
 'Ben Bakes Cakes': 'P08',
 'Ben Is Too Big': 'P08',
 'Big Big Big Bread': 'P08',
 'Big Bob Small Sam': 'P07',
 'Brown Bear`s Birthday': 'P03',
 'Butterflies Fly': 'P08',
 'Cat and Rat Play Together': 'P08',
 'Collin Goes to a Toilet': 'P0

In [192]:
new_par_try = new_par_fu.fillna('nan')
new_par_try

,album_id,sub_title,pr_int
0,749,꼬마버스 타요1,P05
1,750,꼬마버스 타요1,P05
2,2131,꼬마버스 타요1,P05
3,2625,꼬마버스 타요1,P05
4,2594,꼬마버스 타요1,P05
...,...,...,...
39870,39872,로티프렌즈 미술놀이,nan
39871,39873,로티프렌즈 미술놀이,nan
39872,39874,로티프렌즈 미술놀이,nan
39873,4779,4-5세,nan


In [174]:
new_par_try.loc[new_par_try['pr_int'] == 'nan']
# 현재 결측 19514

,album_id,sub_title,pr_int
26,26077,변신자동차 또봇2,NaN
27,26078,변신자동차 또봇2,NaN
28,26079,변신자동차 또봇2,NaN
30,26080,변신자동차 또봇1,NaN
31,13771,변신자동차 또봇1,NaN
...,...,...,...
39870,39872,로티프렌즈 미술놀이,NaN
39871,39873,로티프렌즈 미술놀이,NaN
39872,39874,로티프렌즈 미술놀이,NaN
39873,4779,4-5세,NaN


In [193]:
cnt = 0
for i in new_par_try.itertuples():
    if i[3] == 'nan':
        if i[2] in meta_subtitle_key_dict.keys():
            new_par_try.loc[i[0], 'pr_int'] = meta_subtitle_key_dict[i[2]]
        else:
            cnt += 1

In [194]:
new_par_try

,album_id,sub_title,pr_int
0,749,꼬마버스 타요1,P05
1,750,꼬마버스 타요1,P05
2,2131,꼬마버스 타요1,P05
3,2625,꼬마버스 타요1,P05
4,2594,꼬마버스 타요1,P05
...,...,...,...
39870,39872,로티프렌즈 미술놀이,nan
39871,39873,로티프렌즈 미술놀이,nan
39872,39874,로티프렌즈 미술놀이,nan
39873,4779,4-5세,P01


In [359]:
new_par_try['album_id'].value_counts()

749      1
17056    1
15212    1
12126    1
12124    1
        ..
5407     1
5406     1
32275    1
22598    1
11629    1
Name: album_id, Length: 39875, dtype: int64

In [357]:
meta_genre_with_keyowrd

,album_id,genre_mid,pr_int
0,749,TV만화,P04
1,750,TV만화,P04
2,2131,TV만화,P04
3,2625,TV만화,P04
4,2594,TV만화,P04
...,...,...,...
39850,6302,한글,P08
39851,6303,한글,P08
39852,6309,한글,P08
39853,6310,한글,P08


In [412]:
g_par_try = pd.merge(new_par_try, meta_genre_with_keyowrd[['album_id', 'genre_mid']], on = 'album_id', how = 'left')
g_par_try

,album_id,sub_title,pr_int,genre_mid
0,749,꼬마버스 타요1,P05,TV만화
1,750,꼬마버스 타요1,P05,TV만화
2,2131,꼬마버스 타요1,P05,TV만화
3,2625,꼬마버스 타요1,P05,TV만화
4,2594,꼬마버스 타요1,P05,TV만화
...,...,...,...,...
39870,39872,로티프렌즈 미술놀이,nan,놀이교실
39871,39873,로티프렌즈 미술놀이,nan,놀이교실
39872,39874,로티프렌즈 미술놀이,nan,놀이교실
39873,4779,4-5세,P01,책


In [383]:
for i in g_par_try[0:2].itertuples():
    print(i)

Pandas(Index=0, album_id=749, sub_title='꼬마버스 타요1', pr_int='P05', genre_mid='TV만화')
Pandas(Index=1, album_id=750, sub_title='꼬마버스 타요1', pr_int='P05', genre_mid='TV만화')


In [413]:
genre_cnt = 0
execption = 0
for i in g_par_try.itertuples():
    if i[3] == 'nan':
        genre_cnt += 1
        if i[4] in pr_genre_dict['pr_int'].keys():
            g_par_try.loc[i[0], 'pr_int'] = pr_genre_dict['pr_int'][i[4]]
        else:
            g_par_try.loc[i[0], 'pr_int'] = 'P02'
            print(i)
            execption += 1

Pandas(Index=14296, album_id=32755, sub_title='교육상식', pr_int='nan', genre_mid='휴먼/감동')
Pandas(Index=14297, album_id=32756, sub_title='교육상식', pr_int='nan', genre_mid='휴먼/감동')
Pandas(Index=14298, album_id=32757, sub_title='교육상식', pr_int='nan', genre_mid='휴먼/감동')
Pandas(Index=14299, album_id=32758, sub_title='교육상식', pr_int='nan', genre_mid='휴먼/감동')
Pandas(Index=14300, album_id=32759, sub_title='교육상식', pr_int='nan', genre_mid='휴먼/감동')
Pandas(Index=14301, album_id=32760, sub_title='교육상식', pr_int='nan', genre_mid='휴먼/감동')
Pandas(Index=14302, album_id=32761, sub_title='교육상식', pr_int='nan', genre_mid='휴먼/감동')
Pandas(Index=14303, album_id=32762, sub_title='교육상식', pr_int='nan', genre_mid='휴먼/감동')
Pandas(Index=14304, album_id=32763, sub_title='교육상식', pr_int='nan', genre_mid='휴먼/감동')
Pandas(Index=14305, album_id=32764, sub_title='교육상식', pr_int='nan', genre_mid='휴먼/감동')
Pandas(Index=14306, album_id=32765, sub_title='교육상식', pr_int='nan', genre_mid='휴먼/감동')
Pandas(Index=14307, album_id=32766, sub_tit

In [416]:
pr_key_max = g_par_try[['album_id', 'pr_int']]

In [417]:
pr_key_max

,album_id,pr_int
0,749,P05
1,750,P05
2,2131,P05
3,2625,P05
4,2594,P05
...,...,...
39870,39872,P04
39871,39873,P04
39872,39874,P04
39873,4779,P01


In [418]:
import pickle
with open('pr_key_for_all_item.pickle', 'wb') as f:
    pickle.dump(pr_key_max, f)

In [397]:
genre_cnt

4747

In [411]:
pr_genre_dict['pr_int'].keys()

dict_keys(['MCN', 'TV만화', '극장판 애니', '노래 율동', '노래율동', '놀이교실', '다큐멘터리', '독서동화', '드라마', '수학과학', '시리즈', '시사교양', '애니', '액션/모험', '어린이방송', '역사/문화', '연예오락', '예능', '예술', '외국어', '유아애니', '육아정보', '창의학습', '책', '체육안전', '코미디', '한글', '한자'])

In [410]:
meta_df.genre_mid.value_counts()
# 29개

놀이교실      11084
TV만화      10873
노래율동       7555
외국어        3342
애니         2110
예능         1719
책          1352
창의학습        799
액션/모험       701
유아애니        537
드라마         455
시리즈         390
독서동화        350
육아정보        308
시사교양        212
노래 율동       166
MCN          93
코미디          80
어린이방송        79
다큐멘터리        72
연예오락         62
수학과학         60
한글           60
역사/문화        38
한자           38
예술           21
휴먼/감동        20
극장판 애니       13
체육안전         13
Name: genre_mid, dtype: int64

In [398]:
execption

19

In [387]:
g_par_try

,album_id,sub_title,pr_int,genre_mid
0,749,꼬마버스 타요1,P05,TV만화
1,750,꼬마버스 타요1,P05,TV만화
2,2131,꼬마버스 타요1,P05,TV만화
3,2625,꼬마버스 타요1,P05,TV만화
4,2594,꼬마버스 타요1,P05,TV만화
...,...,...,...,...
39870,39872,로티프렌즈 미술놀이,P04,놀이교실
39871,39873,로티프렌즈 미술놀이,P04,놀이교실
39872,39874,로티프렌즈 미술놀이,P04,놀이교실
39873,4779,4-5세,P01,책


In [369]:
genre_cnt

4728

In [367]:
g_par_try.loc[g_par_try['pr_int'] == 'nan']

,album_id,sub_title,pr_int,genre_mid
